In [11]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict


from rl_env import DraftEnv

How do I compute probabilities of what each player will do?
Thinking about what each player needs.

Factors to consider
 - Starters needed - 0 pr weight on people not needed for starters
 - roll out until next turn - for each player in between self and next pick, apply the same rules
 - i.e. for each player, there say there 
 - 

In [1]:

from collections import deque, namedtuple
from typing import Dict, List
import random
import numpy as np
import pandas as pd
import math



STARTER_COMPOSITION = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DEF": 1, "FLEX": 2}

NUM_DRAFT_ROUNDS = 15
NUM_MGRS = 12

def softmax(x, temperature=1.0):
    x = np.array(x)
    e_x = np.exp((x - np.max(x))/temperature)
    return e_x / e_x.sum(axis=0)


class Draft():
    
    def __init__(self, stochastic_temp):
        
        df_players = self._make_players_df()
        self.stochastic_temp = stochastic_temp
        self.actions = {0: "QB", 1: "RB", 2: "WR", 3: "TE", 4: "K", 5: "DEF"} 
        self.turns = self._make_turns_list()
      
        self.cur_turn = 0  # increments from 0 to NUM_MGRS * NUM_DRAFT_ROUNDS - 1, indexes self.draft to get the current manager
        self.cur_round = 0  # increments from 0 to NUM_DRAFT_ROUNDS - 1
        self.all_players = df_players.dropna(subset=["mean", "std"]).copy()  # all players
        self.all_players = self.all_players.sort_values(by="mean", ascending=False).reset_index(drop=True)
        
        
        self.open_players = self.all_players.copy()  # players that are still available to be drafted
        self.draft = self._make_empty_draft()
        
        self.keepers = {
            0: {"round": 5, "sleeper_id": "8146"},
            1: {"round": 10, "sleeper_id": "2749"},
            2: {"round": 9, "sleeper_id": "9226"},
            3: {"round": 11, "sleeper_id": "8183"},
            4: {"round": 9, "sleeper_id": "1264"},
            5: {"round": 11, "sleeper_id": "5947"},
            6: {"round": 6, "sleeper_id": "8150"},
            7: {"round": 6, "sleeper_id": "10229"},
            8: {"round": 5, "sleeper_id": "6803"},
            9: {"round": 3, "sleeper_id": "2216"},
            10: {"round": 4, "sleeper_id": "5892"},
            11: {"round": 11, "sleeper_id": "10859"}
        }
        
        # remove keepers from open players
        self.open_players = self.open_players.loc[~self.open_players["sleeper_id"].isin([v["sleeper_id"] for v in self.keepers.values()])]
    
    def _make_empty_draft(self):
        draft = pd.DataFrame({  # the draft board
            "round": [turn["round"] for turn in self.turns],
            "mgr": [turn["mgr"] for turn in self.turns],
            "sleeper_id": None,
            "full_name": None,
            "team": None,
            "position": None, # actual position
            "team_pos": None, # labels some players as FLEX
            "fp_mean": None,
            "fp_std": None
        })
        return draft
   
    def get_cur_mgr(self):
        return self.draft["mgr"].values[self.cur_turn]
    
    def get_needed_pos_counts(self, mgr_num: int, flex=True) -> Dict[str, int]:
        ''' Get the number of each position needed for a manager to fill their starters '''
        team_comp = self.get_team_comp(mgr_num, flex=flex)
        needed_pos_counts = {pos: STARTER_COMPOSITION[pos] - team_comp[pos] for pos in STARTER_COMPOSITION}
        return needed_pos_counts
    

    def stochastic_choice(self, mgr_num: int=None, temperature=None) -> dict:
        '''
        picks a random (intelligent) player for the NPC managers.
        unlike reasonable_action methods, this returns the player as a dict
        This allows us to choose players who are not the highest ranked player in a position
        '''
        if temperature is None:
            temperature = self.stochastic_temp
        if mgr_num is None:
            mgr_num = self.get_cur_mgr()
        team_comp = self.get_team_comp(mgr_num, flex=True)
        needed_pos_counts = self.get_needed_pos_counts(mgr_num, flex=True)
        
        # first get what non-flex positions are needed
        needed_positions = [pos for pos, count in needed_pos_counts.items() if pos != "FLEX" and count > 0]
        rb_te_wr_filled = team_comp.get("RB", 0) >= STARTER_COMPOSITION["RB"] and \
            team_comp.get("WR", 0) >= STARTER_COMPOSITION["WR"] and \
            team_comp.get("TE", 0) >= STARTER_COMPOSITION["TE"]
        
        # if the rb, wr, and te are filled, check flex
        if needed_pos_counts.get("FLEX", 0) > 0 and rb_te_wr_filled:
            # don't worry about flex if rb, wr, te are not filled yet
                needed_positions += ["RB", "WR", "TE"]
                
        # if we need players, choose the highest point player available of any needed position
        if len(needed_positions) != 0:
            needed_players = self.open_players[self.open_players['position'].isin(needed_positions)]
            if not needed_players.empty:
                # if players are available in a needed position, choose the highest point player in a needed position
                # select up to 5 of the top players, with a max of 100 mean less than the top
                options = needed_players.loc[needed_players["mean"] >= needed_players["mean"].max() - 100]
                options = options.iloc[:np.min([5, len(options)])]
                chosen_player = options.sample(1, weights=softmax(options["mean"].values, temperature=temperature)).iloc[0]
                
            else:
                # if no players are available in a needed position, choose the highest point player available
                # chosen_player = self.open_players.iloc[0]
                options = self.open_players.loc[self.open_players["mean"] >= self.open_players["mean"].max() - 100]
                options = options.iloc[:np.min([5, len(options)])]
                chosen_player = options.sample(1, weights=softmax(options["mean"].values, temperature=temperature)).iloc[0]
                
        else: 
            # if we don't need players, choose the highest point player available
            # chosen_player = self.open_players.iloc[0]
            options = self.open_players.loc[self.open_players["mean"] >= self.open_players["mean"].max() - 100]
            options = options.iloc[:np.min([5, len(options)])]
            chosen_player = options.sample(1, weights=softmax(options["mean"].values, temperature=temperature)).iloc[0]
            
        return chosen_player.to_dict()

    def reasonable_action(self, mgr_num: int=None) -> int:
        ''' Choose a reasonable option based on the team composition of a manager.
        The reasonable option is the highest point player in a position for which the manager
        still needs to fill a starting position. Else, choose highest point player available.
        '''
        if mgr_num is None:
            mgr_num = self.get_cur_mgr()
        team_comp = self.get_team_comp(mgr_num, flex=True)
        needed_pos_counts = self.get_needed_pos_counts(mgr_num) # includes flex
        
        # first get what non-flex positions are needed
        needed_positions = [pos for pos, count in needed_pos_counts.items() if pos != "FLEX" and count > 0]
        rb_te_wr_filled = team_comp.get("RB", 0) >= STARTER_COMPOSITION["RB"] and \
            team_comp.get("WR", 0) >= STARTER_COMPOSITION["WR"] and \
            team_comp.get("TE", 0) >= STARTER_COMPOSITION["TE"]
        
        # if the rb, wr, and te are filled, check flex
        if needed_pos_counts.get("FLEX", 0) > 0 and rb_te_wr_filled:
            # don't worry about flex if rb, wr, te are not filled yet
                needed_positions += ["RB", "WR", "TE"]
                
        # if we need players, choose the highest point player available of any needed position
        if len(needed_positions) != 0:
            needed_players = self.open_players[self.open_players['position'].isin(needed_positions)]
            if not needed_players.empty:
                # if players are available in a needed position, choose the highest point player in a needed position
                chosen_player = needed_players.iloc[0]
            else:
                # if no players are available in a needed position, choose the highest point player available
                chosen_player = self.open_players.iloc[0]
        else: 
            # if we don't need players, choose the highest point player available
            chosen_player = self.open_players.iloc[0]
        action = [num for num, pos in self.actions.items() if pos == chosen_player["position"]][0]
        return action
    
    def reasonable_action_stoch(self, mgr_num: int=None, temperature: float=.1) -> int:
        ''' Choose a random reasonable option based on the team composition of a manager.
        Note this function doesn't exactly work right, because it ultimately chooses a position
        that is needed, but then chooses the highest point player in that position. 
        '''
        if mgr_num is None:
            mgr_num = self.get_cur_mgr()
        choice = self.stochastic_choice(mgr_num, temperature=temperature)
        return choice["position"]
        
        
    def get_team_comp(self, mgr_num: int, flex=True) -> Dict[str, int]:
        ''' Get the team composition for a manager.
        If flex is True, then the FLEX position is included in the count'''
        if flex:
            team_comp = self.draft.loc[(self.draft["mgr"] == mgr_num)].groupby("team_pos").size().to_dict()
            team_comp["FLEX"] = team_comp.get("FLEX", 0)
        else:
            team_comp = self.draft.loc[(self.draft["mgr"] == mgr_num)].groupby("position").size().to_dict()
        for pos in ["QB", "RB", "WR", "TE", "K", "DEF"]:
            # fill in missing positions with 0
            team_comp[pos] = team_comp.get(pos, 0)
        return team_comp
    
    def _make_turns_list(self):
        turns = []
        for round_num in range(15):
            if round_num % 2 == 0:  # Even numbered rounds (0, 2, 4, ...)
                for i in range(12):
                    turns.append({"round": round_num, "mgr": i})
            else:  # Odd numbered rounds (1, 3, 5, ...)
                for i in range(11, -1, -1):
                    turns.append({"round": round_num, "mgr": i})
        return turns
        
        
    def reset(self, seed=None):
        """Resets the environment to an initial state and returns an initial observation."""
        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
        
        self.draft = self._make_empty_draft()
        self.cur_round = 0
        self.cur_turn = 0
        self.open_players = self.all_players.copy()
    
        return True
    
    def is_starters_filled(self, mgr_num: int) -> bool:
        filled = [x >= STARTER_COMPOSITION[pos] for pos, x in self.get_team_comp(mgr_num, flex=True).items()]
        if all(filled):
            return True
        return False
    
    def get_starters(self, mgr_num: int=None) -> pd.DataFrame:
        if mgr_num is None:
            mgr_num = self.get_cur_mgr()
        team = self.draft.loc[(self.draft["mgr"] == mgr_num)].copy()
        starter_ids = []
        for pos in STARTER_COMPOSITION.keys():
            players_pos = team.loc[team["team_pos"] == pos]
            players_pos = players_pos.sort_values(by=self.objective, ascending=False).reset_index(drop=True)
            pos_starter_ids = players_pos.iloc[:STARTER_COMPOSITION[pos]]['sleeper_id'].values
            if len(pos_starter_ids) > 0:
                starter_ids += list(pos_starter_ids)
        return team.loc[team["sleeper_id"].isin(starter_ids)]
        
            
    def get_bench(self, mgr_num: int=None) -> pd.DataFrame:
        if mgr_num is None:
            mgr_num = self.get_cur_mgr()
        team = self.draft.loc[(self.draft["mgr"] == mgr_num)].copy()
        starter_ids = self.get_starters(mgr_num)['sleeper_id'].values
        return team.loc[~team["sleeper_id"].isin(starter_ids)]
    
    def get_team(self, mgr_num: int) -> pd.DataFrame:
        return self.draft.loc[self.draft["mgr"] == mgr_num]
    
    def get_mgr_rankings(self, starters=True) -> pd.DataFrame:
        # identify which rounds are all full
        # full_rounds = self.draft.groupby('round')['fp_mean'].apply(lambda x: x.notnull().all())
        # full_rounds = full_rounds[full_rounds].index
        # draft = self.draft[self.draft['round'].isin(full_rounds)]
        if starters:
            starter_draft = pd.concat([self.get_starters(mgr_num) for mgr_num in range(NUM_MGRS)])
            rankings = starter_draft.groupby('mgr')['fp_mean'].sum().sort_values(ascending=False)
        else:
            rankings = self.draft.groupby('mgr')['fp_mean'].sum().sort_values(ascending=False)
        rankings = pd.DataFrame({
            "rank": np.arange(1, NUM_MGRS+1),
            "mgr": rankings.index,
            "fp_mean": rankings.values
            }).reset_index(drop=True)
        return rankings
    
    def choose_player(self, mgr_num: int, sleeper_id: str):
        '''updates draft and open players'''
        assert sleeper_id in self.open_players["sleeper_id"].values
   
        player = self.open_players.loc[self.open_players["sleeper_id"] == sleeper_id].iloc[0].to_dict()
        
        # ignore choice and use keeper if it is the keeper round for the manager
        keeper_round = self.keepers[mgr_num]['round']
        if int(keeper_round) == int(self.cur_round):
            # print(f"Keeper round for mgr {mgr_num}")
            player_row = self.all_players.loc[self.all_players['sleeper_id'] == self.keepers[mgr_num]['sleeper_id']]
            player = {
                "sleeper_id": player_row['sleeper_id'].values[0],
                "full_name": player_row['full_name'].values[0],
                "team": player_row['team'].values[0],
                "position": player_row['position'].values[0],
                "mean": player_row['mean'].values[0],
                "std": player_row['std'].values[0]
            }
        
        team_comp = self.get_team_comp(mgr_num, flex=False)
        # Indicates if the team is ready to draft a flex player, ie. rb, wr, te are filled
        flex_ready = team_comp.get("RB") >= STARTER_COMPOSITION["RB"] and \
            team_comp.get("WR") >= STARTER_COMPOSITION["WR"] and \
            team_comp.get("TE") >= STARTER_COMPOSITION["TE"]
        # If mgr is ready for flex player, then assign FLEX
        if player["position"] in ["RB", "WR", "TE"] and flex_ready:
            player["team_pos"] = "FLEX"
        else: 
            player["team_pos"] = player["position"]
        
        # -- Add the player to the draft -- #
        self.draft.loc[self.cur_turn, "sleeper_id"] = str(player["sleeper_id"])
        self.draft.loc[self.cur_turn, "full_name"] = str(player["full_name"])
        self.draft.loc[self.cur_turn, "team"] = str(player["team"])
        self.draft.loc[self.cur_turn, "position"] = str(player["position"])
        self.draft.loc[self.cur_turn, "team_pos"] = str(player["team_pos"])
        self.draft.loc[self.cur_turn, "fp_mean"] = float(player["mean"])
        self.draft.loc[self.cur_turn, "fp_std"] = float(player["std"])
        
        # -- Remove the player from the open players -- #
        self.open_players = self.open_players.loc[self.open_players["sleeper_id"] != sleeper_id]
        
        return player
    
        
    
    def increment_turn(self):
        '''updates the state, increments the turn, and updates the round'''
        # self.update_state()
        
        self.cur_turn += 1
        if self.cur_turn < len(self.turns):
            self.cur_round = self.draft["round"].values[self.cur_turn]
        else:
            # even though this round doesn't exist,
            # we need to have a valid int for the state
            self.cur_round = NUM_DRAFT_ROUNDS
        
        return True
    
   
    
    def draft_full(self) -> bool:
        return self.draft["fp_mean"].isnull().sum() == 0
    
    def get_mgr_draft(self, mgr_num: int) -> pd.DataFrame:
        return self.draft.loc[self.draft["mgr"] == mgr_num]
    
    def get_state(self):
        return self.state
    
    def get_sum_fp(self, mgr_num: int, starters=False) -> float:

        
        # If 'starters' is True, filter the DataFrame to include only the starters
        if starters:
            team = self.get_starters(mgr_num)
        else: 
            team = self.get_team(mgr_num)
            
        sum_fp = team["fp_mean"].sum()
    
        return sum_fp
    
    def _make_players_df(self):
        df_sleeper = pd.read_csv("data/sleeper/all_players.csv")
        df_qb_proj = pd.read_csv("data/projections/QB_projections.csv")
        df_rb_proj = pd.read_csv("data/projections/RB_projections.csv")
        df_wr_proj = pd.read_csv("data/projections/WR_projections.csv")
        df_te_proj = pd.read_csv("data/projections/TE_projections.csv")
        df_k_proj = pd.read_csv("data/projections/K_projections.csv")
        df_def_proj = pd.read_csv("data/projections/DEF_projections.csv")



        df_qb_proj = df_qb_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_rb_proj = df_rb_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_wr_proj = df_wr_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_te_proj = df_te_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_k_proj = df_k_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)
        df_def_proj = df_def_proj.loc[:, ["sleeper_id", "full_name", "team", "position", "source", "fpts"]].sort_values(by="fpts", ascending=False)

        df_proj = pd.concat([df_qb_proj, df_rb_proj, df_wr_proj, df_te_proj, df_k_proj, df_def_proj])

        df_proj_agg = df_proj.groupby('sleeper_id')['fpts'].agg(['mean', 'std']).reset_index()
        df_proj_agg['sleeper_id'] = df_proj_agg['sleeper_id'].astype(str)


        df_players = df_proj_agg.merge(df_sleeper.loc[:, ['sleeper_id', 'full_name', 'position', 'team']], 
                                        on='sleeper_id', 
                                        how='left')
        return df_players

    
env = Draft(stochastic_temp=1.0)


In [ ]:
import pymc as pm
import numpy as np


# Run the simulation
with pm.Model() as model:
    for round_num in range(NUM_DRAFT_ROUNDS):
        print(f"Round {round_num + 1}")
        
        for manager in managers:
            pick = manager.stochastic_choice()
            trace = pm.sample_prior_predictive(samples=1)
            
            # Accessing the prior group
            prior_samples = trace.prior
            
            # Retrieve the sampled index and ensure it's an integer
            picked_player_idx = int(prior_samples[f'{manager.name}_pick'].values[0])
            picked_player = manager.players[picked_player_idx]
            print(f"{manager.name} picked {picked_player}")
            manager.pick_history.append(picked_player)
            
            # Remove the picked player from all managers' player pools
            for m in managers:
                m.players = np.delete(m.players, picked_player_idx)
                m.positions = np.delete(m.positions, picked_player_idx)
                m.player_scores = np.delete(m.player_scores, picked_player_idx)

        print()

# Review the picks made by each manager
for manager in managers:
    print(f"{manager.name}'s picks: {manager.pick_history}")


In [90]:
players = Players()
ashish = Manager(1, players)
ashish.stochastic_choice()


[5.86889370e-20 5.82638582e-15 1.87497321e-02 7.34786457e-20
 9.81250268e-01]


{'sleeper_id': '4034',
 'mean': 335.53424699283335,
 'std': 25.370946800430414,
 'full_name': 'Christian McCaffrey',
 'position': 'RB',
 'team': 'SF'}